purpose: run network propagation for a given dataset, in this case used for locomotor activity and externalizing.

In [1]:
import os
import pandas as pd
import ndex2
import networkx as nx
from netcoloc import netprop_zscore
from netcoloc import netprop
from netcoloc import network_colocalization
import sys
import random
#os.chdir('/tscc/projects/ps-palmer/brittany/rare_common_alcohol/rare_common_alcohol_comparison/notebooks/')
#from rca_functions import *
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/scripts')
from network_functions import *
from network_validation_functions import *
from plotting_functions import *
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/')

In [2]:
random_seed=random.seed(211)

In [3]:
save_file=True

In [4]:
#create a file called environ_ndex_meta.py where you save variables 'ndex_user' and 'ndex_password'
#otherwise will prompt you to define those within the notebooks
if os.path.isfile('../environ_ndex_meta.py'):
    print ('NDEx credentials imported from meta file')
    sys.path.insert(1, '../')
    from environ_ndex_meta import *
    sys.path.pop(1)
else:
    # Prompt the user for a username
    ndex_user = input("Enter your NDEx username: ")
    # Prompt the user for a password
    ndex_password = input("Enter your NDEx password: ")

NDEx credentials imported from meta file


# functions

In [5]:
#from rat bmi notebooks not netcoloc
def calculate_heat_zscores_with_sampling(data, nodes, individual_heats, G_PC, trait="BMI", max_genes=500, num_samples=100,
                                        nominal_sig=0.05, num_reps=1000, out_path="", minimum_bin_size=10,outfile='sample.tsv'):
    """Takes a set of summary statistics and a molecular interaction and performs sampling of the significant genes.
    For each sample a random selection of seed genes is chosen, weighted by the p-value of each gene in the summary
    statistics. Network propagation with zscore calculation is performed for each sample to generate a distribution
    of z-scores for each gene in the seed_gene set.

    Args:
        data (pd.DataFrame): Gene level summary statistics
        nodes (list): list of nodes in the interaction network
        individual_heats (np.array): Heat matrix calculated by `netprop_zscore.get_individual_heats_matrix()`
        G_PC (nx.Graph): molecular interaction network
        trait (str, optional): name of trait being investigated. Defaults to "BMI".
        max_genes (int, optional): Maximum number of seed genes to include in each sample (maximum=500). Defaults to 500.
        num_samples (int, optional): Number of times to perform sampling. Defaults to 100.
        nominal_sig (float, optional): Significance cutoff for keeping genes in data (Note: this value will be Bonferroni corrected). Defaults to 0.05.
        num_reps (int, optional): Number of repetitions of randomization for generating null distribution for z_scores. Defaults to 1000.
        out_path (str, optional): File path prefix for saving results of sampling. Defaults to "".
        minimum_bin_size (int, optional): minimum number of genes that should be in each degree matching bin. Defaults to 10.

    Returns:
        pd.DataFrame: Gene x sampling run dataframe of sampled z-scores
    """
    assert max_genes <= 500, "NetColoc is only valid for sets of 500 or less genes so maximum number of genes for sampling must be <= 500"
    #outfile = out_path + trait + "sampling_" + str(max_genes) + "_" + str(num_samples) + ".tsv"
    data = data.loc[data.gene_symbol.isin(nodes)]  # subset to genes in interaction network
    all_seeds = data.loc[data.pvalue <= nominal_sig / len(data)]  # Bonferroni correction
    all_seeds = all_seeds.assign(log10p=-1 * np.log10(all_seeds.pvalue))  # get -log10p for weighted sampling
    sampling_results = []
    for i in tqdm(range(num_samples)):
        # perform propagation for sample
        sample_seeds = random.choices(population=all_seeds.gene_symbol.values, weights=all_seeds.log10p.values, k=max_genes)
        sample_results = netprop_zscore.calculate_heat_zscores(individual_heats, nodes=list(G_PC.nodes), degrees=dict(G_PC.degree),
                                                seed_genes=sample_seeds, num_reps=num_reps,
                                                minimum_bin_size=minimum_bin_size, random_seed=i)[0]
        sample_z = pd.DataFrame(sample_results, columns=["z" + str(i)])
        # save running results of sampling
        if i == 0:
            sample_z.to_csv(outfile, sep="\t")
        else:
            existing = pd.read_csv(outfile, sep="\t", index_col=0)
            existing = existing.join(sample_z)
            existing.to_csv(outfile, sep="\t")
        sampling_results.append(sample_z)
    return pd.concat(sampling_results, axis=1)

# Interactome Set-up

pcnet2- versions 
from wright et al. 2024 preprint:
PCNet 2.0= best-performing ranked composite (top 15 interactomes, 3.85M interactions)
PCNet 2.1= top 8 interactomes, 1.75M interactions
PCNet 2.2= top 10 co-citation-free interactomes, 3.32M interactions 

In [5]:
#choose which interactome you want to import
tissue_network=False

if tissue_network:
    tissue='basal_ganglion'
else:
    interactome_name='PCNet2.0'

In [6]:
if not tissue_network:
    interactome=import_interactome(UUIDs=UUIDs,interactome_name=interactome_name)
    all_nodes=list(interactome.nodes())
    # pre calculate the matricies used for network propagation
    print('\ncalculating w_prime')
    w_prime = netprop.get_normalized_adjacency_matrix(interactome, conserve_heat=True)
    print('\ncalculating w_double_prime')
    w_double_prime = netprop.get_individual_heats_matrix(w_prime, .5)
    edges=list(interactome.edges())
    all_nodes=list(interactome.nodes())
    degree=interactome.degree()
else:
    netdir='tissue_networks/intermediate/'
    
    print(f'available files for this tissue: {[x for x in os.listdir(netdir) if tissue in x]}')
    interactome_name=f'hb_tissue_{tissue}_top'
    #import node list
    with open(f'{netdir}node_list_{tissue}_top.txt', 'r') as file:
        lines = file.readlines()
    # Remove newline characters from each line
    all_nodes=[line.strip() for line in lines]
    #import degrees
    degree=pd.read_csv(f'{netdir}degree_{tissue}_top.csv', header=None)
    degree.index=degree[0].astype(str)
    degree=degree[1].to_dict()

    print(f'importing files for {tissue}_top')
    #import w_double_prime
    if os.path.exists(f'{netdir}w_double_prime_{tissue}_top.npy'):
        print('importing w_double_prime from file')
        w_double_prime=np.load(f'{netdir}w_double_prime_{tissue}_top.npy')
    elif os.path.exists(f'{netdir}normalized_adjacency_{tissue}_top.npz'):
        print('w_double_prime not calculated previously- importing w_prime, from which w_double_prime will be calculated')
        w_prime=sp.load_npz(f'{netdir}normalized_adjacency_{tissue}.npz')
        w_double_prime=netprop.get_individual_heats_matrix(w_prime, .5)
        np.save(f'{outdir}w_double_prime_{tissue}',w_double_prime)
    else:
        print('files not found- interactome files must be calculated before continuing')

PCNet2.0
Name: PCNet 2.0
Nodes: 19267
Edges: 3852119
Node Attributes: 19267
Edge Attributes: 19260595

number of nodes:
19267

number of edges:
3852119

calculating w_prime

calculating w_double_prime


# import seed genes

In [7]:
if not tissue_network:
    seed_dict=import_seed_dict(mag_dir,file_dict,ctrl_traits,ctrl_traits_rat,bonf_dict,gene_col_dict,all_nodes)
else:
    hgnc=pd.read_csv('hgnc_complete_set.txt',sep='\t',low_memory=False)
    hgnc=hgnc[['symbol','entrez_id']].dropna()
    hgnc['entrez_id']=hgnc['entrez_id'].astype(int).astype(str)
    seed_dict=import_seed_dict(mag_dir,file_dict,bonf_dict,gene_col_dict,hgnc[hgnc.entrez_id.isin(all_nodes)]['symbol']) 
seed_dict.keys()

dict_keys(['loco_bonf', 'loco_top500', 'loco_FDR', 'loco_gsem_bonf', 'loco_gsem_top500', 'loco_gsem_FDR', 'ext_bonf', 'ext_top500', 'ext_FDR', 'ext_st22', 'loco_mega_fus_naac_bonf', 'loco_mega_fus_naac_top500', 'loco_mega_fus_naac_FDR', 'ext_fus_naac_bonf', 'ext_fus_naac_top500', 'ext_fus_naac_FDR', 'loco_final_cf_bonf', 'loco_final_cf_top500', 'loco_final_cf_FDR', 'loco_final_mega_bonf', 'loco_final_mega_top500', 'loco_final_mega_FDR', 'ext_rat_bonf', 'ext_rat_top500', 'ext_rat_FDR', 'loco_final_cf_rat_bonf', 'loco_final_cf_rat_top500', 'loco_final_cf_rat_FDR', 'ext_db_bonf', 'ext_db_top500', 'ext_db_FDR', 'ext_rtb_bonf', 'ext_rtb_top500', 'ext_rtb_FDR', 'loco_final_cf_25_bonf', 'loco_final_cf_25_top500', 'loco_final_cf_25_FDR', 'facial_hair_FDR', 'facial_hair_bonf', 'facial_hair_top500', 'age_smkinit_FDR', 'age_smkinit_bonf', 'age_smkinit_top500', 'antisoc_FDR', 'antisoc_bonf', 'antisoc_top500', 'friend_sat_FDR', 'friend_sat_bonf', 'friend_sat_top500', 'hr_FDR', 'hr_bonf', 'hr_top500

In [8]:
for t in seed_dict.keys():
    print(f'{t}: {len(seed_dict[t])}')

loco_bonf: 11
loco_top500: 498
loco_FDR: 376
loco_gsem_bonf: 30
loco_gsem_top500: 497
loco_gsem_FDR: 458
ext_bonf: 1106
ext_top500: 500
ext_FDR: 5044
ext_st22: 34
loco_mega_fus_naac_bonf: 4
loco_mega_fus_naac_top500: 497
loco_mega_fus_naac_FDR: 140
ext_fus_naac_bonf: 49
ext_fus_naac_top500: 500
ext_fus_naac_FDR: 134
loco_final_cf_bonf: 9
loco_final_cf_top500: 492
loco_final_cf_FDR: 342
loco_final_mega_bonf: 26
loco_final_mega_top500: 493
loco_final_mega_FDR: 640
ext_rat_bonf: 980
ext_rat_top500: 19
ext_rat_FDR: 4373
loco_final_cf_rat_bonf: 9
loco_final_cf_rat_top500: 20
loco_final_cf_rat_FDR: 338
ext_db_bonf: 81
ext_db_top500: 500
ext_db_FDR: 479
ext_rtb_bonf: 318
ext_rtb_top500: 500
ext_rtb_FDR: 2002
loco_final_cf_25_bonf: 9
loco_final_cf_25_top500: 489
loco_final_cf_25_FDR: 373
facial_hair_FDR: 763
facial_hair_bonf: 159
facial_hair_top500: 500
age_smkinit_FDR: 33
age_smkinit_bonf: 8
age_smkinit_top500: 500
antisoc_FDR: 0
antisoc_bonf: 0
antisoc_top500: 500
friend_sat_FDR: 8
friend_sa

In [68]:
rat_TWAS_tissue_label={'BLA':'Basolateral amygdala',
'Brain':'Brain hemisphere',
'IL':'Infralimbic cortex',
'LHb':'Lateral habenula',
'NAcc':'Nucleus accumbens core',
'NAcc1':'Nucleus accumbens core 1',
'NAcc2':'Nucleus accumbens core 2',
'OFC':'Orbitofrontal cortex',
'PL':'Prelimbic cortex',
'PL1':'Prelimbic cortex 1',
'PL2':'Prelimbic cortex 2',
'Adipose':'Adipose',
'Eye':'Eye',
'Liver':'Liver'}

prefix='loco_final_CF'
if not tissue_network: 
    for t in rat_TWAS_tissue_label.keys():
        path=f'rat_fusion/output/FUSION_concat/{prefix}_{t}_seed_genes.dat'
        if os.path.exists(path):
            tbl=pd.read_csv(path,low_memory=False, sep='\t')
            seed_dict[f'{prefix.lower()}_{t}_bonf']=set(tbl[tbl['TWAS.P']<0.05/len(tbl)]['HM_ORTHO'].dropna())
            seed_dict[f'{prefix.lower()}_{t}_FDR']=set(tbl[tbl['Q']<0.05]['HM_ORTHO'].dropna())
            tbl=tbl[~tbl.HM_ORTHO.isna()]
            seed_dict[f'{prefix.lower()}_{t}_top500']=set(tbl[tbl['HM_ORTHO'].isin(all_nodes)].nsmallest(500,'TWAS.P')['HM_ORTHO'])
            print(f'{t} (NORTHO={len(set(tbl.HM_ORTHO))}) BONF: {len(seed_dict[f"{prefix.lower()}_{t}_bonf"])} FDR: {len(seed_dict[f"{prefix.lower()}_{t}_FDR"])} top500: {len(seed_dict[f"{prefix.lower()}_{t}_top500"])}')
        else:
           print(f'{path} does not exist')  

BLA (NORTHO=5228) BONF: 11 FDR: 170 top500: 495
Brain (NORTHO=7767) BONF: 16 FDR: 206 top500: 497
IL (NORTHO=3226) BONF: 7 FDR: 106 top500: 491
LHb (NORTHO=2802) BONF: 5 FDR: 75 top500: 492
NAcc (NORTHO=5490) BONF: 8 FDR: 154 top500: 498
NAcc1 (NORTHO=2887) BONF: 5 FDR: 104 top500: 494
NAcc2 (NORTHO=4659) BONF: 9 FDR: 122 top500: 496
OFC (NORTHO=3247) BONF: 8 FDR: 89 top500: 492
PL (NORTHO=6421) BONF: 15 FDR: 162 top500: 495
PL1 (NORTHO=3278) BONF: 9 FDR: 101 top500: 493
PL2 (NORTHO=5530) BONF: 5 FDR: 184 top500: 494
Adipose (NORTHO=7119) BONF: 7 FDR: 155 top500: 492
Eye (NORTHO=989) BONF: 3 FDR: 14 top500: 496
Liver (NORTHO=6216) BONF: 12 FDR: 190 top500: 492


In [37]:
if tissue_network:
    seed_dict={k: set(hgnc[hgnc.symbol.isin(v)]['entrez_id']) for k, v in seed_dict.items()}

In [15]:
#NPS_dict,NPS_dict_series=import_NPS_scores(seed_dict,interactome_name)

# run straight propagation- no sampling

In [70]:
ls=[i for i in seed_dict.keys() if 'loco_final_cf' in i]

In [9]:
overwrite=False

In [12]:
rm=['cud','alc_dep','loco','ext'] # remove these because WAY too few seed genes for these traits
ls=[x for x in seed_dict.keys() if not any(rm in x for rm in rm)]

In [13]:
ls

['facial_hair_FDR',
 'facial_hair_bonf',
 'facial_hair_top500',
 'age_smkinit_FDR',
 'age_smkinit_bonf',
 'age_smkinit_top500',
 'antisoc_FDR',
 'antisoc_bonf',
 'antisoc_top500',
 'friend_sat_FDR',
 'friend_sat_bonf',
 'friend_sat_top500',
 'hr_FDR',
 'hr_bonf',
 'hr_top500',
 'infant_bw_FDR',
 'infant_bw_bonf',
 'infant_bw_top500',
 'LDL_FDR',
 'LDL_bonf',
 'LDL_top500',
 'maternal_smok_FDR',
 'maternal_smok_bonf',
 'maternal_smok_top500',
 'age_menarche_FDR',
 'age_menarche_bonf',
 'age_menarche_top500',
 'addict-rf_FDR',
 'addict-rf_bonf',
 'addict-rf_top500',
 'adhd_FDR',
 'adhd_bonf',
 'adhd_top500',
 'dpw_FDR',
 'dpw_bonf',
 'dpw_top500',
 'risk_FDR',
 'risk_bonf',
 'risk_top500',
 'auto_speed_FDR',
 'auto_speed_bonf',
 'auto_speed_top500',
 'nsex_FDR',
 'nsex_bonf',
 'nsex_top500',
 'bmi_FDR',
 'bmi_bonf',
 'bmi_top500',
 'height_FDR',
 'height_bonf',
 'height_top500',
 'bmi_rn6_FDR',
 'bmi_rn6_bonf',
 'bmi_rn6_top500',
 'body_length_rn6_FDR',
 'body_length_rn6_bonf',
 'body_le

In [17]:
ls=[x+'_bonf' for x in ctrl_traits] #rerun because of parenthesis issue in bonferroni definition for seed genes- should be the same but doing this to confirm

In [19]:
overwrite=True

In [26]:
#loop for only subset of traits genes
#ls=[i for i in seed_dict.keys() if 'final' in i]
#ls=seed_dict.keys()
for k in ls:  
    seed_genes = list(seed_dict[k].intersection(all_nodes))
    file_path='network_scores/'+k+'_'+interactome_name+'_zscore.tsv'
    print(file_path)
    if ((os.path.exists(file_path))&(overwrite==False)):
        print('File already exists. If you would like to overwrite this file, set overwrite=True, and rerun')
    else:
        NPSc, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
            w_double_prime,  
            list(all_nodes),
            dict(degree), 
            seed_genes, num_reps=1000,
            minimum_bin_size=100,
            random_seed=random_seed)
        print(NPSc.head())
        print('writing to file')
        NPSc.to_csv(file_path,sep='\t',header=False)

network_scores/facial_hair_bonf_PCNet2.0_zscore.tsv


  0%|          | 0/1000 [00:00<?, ?it/s]

TGFBR1    0.627309
TGFBR2    1.581910
SMAD3     0.053780
TGFB1     0.710630
SMAD2     0.380362
dtype: float64
writing to file
network_scores/age_smkinit_bonf_PCNet2.0_zscore.tsv


  0%|          | 0/1000 [00:00<?, ?it/s]

TGFBR1   -0.356040
TGFBR2   -0.205673
SMAD3    -0.586610
TGFB1     1.434154
SMAD2    -1.020521
dtype: float64
writing to file
network_scores/antisoc_bonf_PCNet2.0_zscore.tsv


...

TGFBR1   -0.996836
TGFBR2    2.507146
SMAD3    -0.706087
TGFB1     1.599001
SMAD2     0.491384
dtype: float64
writing to file
network_scores/nsex_bonf_PCNet2.0_zscore.tsv


  0%|          | 0/1000 [00:00<?, ?it/s]

TGFBR1    0.160999
TGFBR2    1.972221
SMAD3    -0.032451
TGFB1     0.390345
SMAD2     1.428497
dtype: float64
writing to file
network_scores/bmi_bonf_PCNet2.0_zscore.tsv


  0%|          | 0/1000 [00:00<?, ?it/s]

TGFBR1    0.540145
TGFBR2    2.580657
SMAD3     2.930848
TGFB1     0.956608
SMAD2     1.467553
dtype: float64
writing to file


# run for an individual seed gene set

In [14]:
overwrite=False

In [27]:
k='loco_final_cf_25_bonf'
print(len(seed_dict[k]))
seed_genes = list(seed_dict[k].intersection(all_nodes))

NPSc, Fnew_score, Fnew_rand_score = netprop_zscore.calculate_heat_zscores(
    w_double_prime,  
    list(all_nodes),
    dict(degree), 
    seed_genes, num_reps=1000,
    minimum_bin_size=100,
    random_seed=random_seed)
print(NPSc.head())
if save_file:
    file_path='network_scores/'+k+'_'+interactome_name+'_zscore.tsv'
    print(file_path)
    if ((overwrite==False)&(os.path.exists(file_path))):
        print('File already exists. If you would like to overwrite this file, set overwrite=True, and rerun')
    else:
        NPSc.to_csv(file_path,sep='\t',header=False)

9


  0%|          | 0/1000 [00:00<?, ?it/s]

TGFBR1   -0.570225
TGFBR2   -0.131549
SMAD3     0.386757
TGFB1    -0.176857
SMAD2     0.195055
dtype: float64
network_scores/loco_final_cf_25_bonf_PCNet2.0_zscore.tsv
